### Summary

작성일 : 19/08/15  
참고 : 202_Find_Users_by_V

<목적>  
client_id를 추출할 수 있는 새로운 조합 찾기
***202_Find_Users_by_V*** 버전의 실제 코드 구현을 통한 client 식별 코드


<분석 배경>  
지금까지 client_id를 TransactionAmt(거래액), Card1, Deviceinfo가 동일한 건으로 묶어서 봐왔음.  
이 방법은 같은 고객이 다른 카드를 쓸 수도 있고, 거래액도 각기 다를 수 있다는 점에서 한계가 있음

<진행 방법>  
V 변수 중 count가 증가하는 변수(ex 1, 2, 3, ...)와 transactionAmt를 더해서 증가하는 변수(ex 가격이 20이라면 20, 40, 60 , 80 ...)  
를 통해 같은 유저로 추정되는 사람을 묶기


<향후 계획>
- 리팩토링 작업을 통해 코드 속도 향상 -> 전체 데이터로 묶기
- 같은 client id들의 특성 찾기, 이를 기반으로 한 train/test의 현재 식별 불가능한 '동일'한 client_id를 찾기 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc, datetime, time
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
from plotnine import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_trans = pd.read_csv("/Users/jeong/downloads/IEEE/input/train_transaction.csv")
train_idf = pd.read_csv("/Users/jeong/downloads/IEEE/input/train_identity.csv")

test_trans = pd.read_csv("/Users/jeong/downloads/IEEE/input/test_transaction.csv")
test_idf = pd.read_csv("/Users/jeong/downloads/IEEE/input/test_identity.csv")

train_trans = pd.merge(train_trans, train_idf, on = 'TransactionID', how = 'left')
test_trans = pd.merge(test_trans, test_idf, on = 'TransactionID', how = 'left')

id를 묶는 방법은 다음과 같습니다.

1. 기존 데이터에서 V95의 최댓값 'temp_n'을 선택합니다. (ex. temp_n = 880)
2. temp_n보다 1 작은 값들의 집합들인 'temp_n_1'을 선택합니다. (ex. temp_n_1 = 879 인 값들의 집합)
3. temp_n_1 중 temp_n['V126'] 와 temp_n_1['V126']의 차가 transactionAmt와 같은 값을 찾습니다.
4. 3.의 과정에서 매칭되는 값이 있으면 temp_n을 기존 데이터에서 삭제하고, 매칭되는 temp_n_1을 temp값으로 치환합니다.  
(이때 다음 과정의 1.은 생략하고 2.부터 작업을 다시 시작합니다)
5. 만약 매칭되는 값이 없거나, temp_n_1의 길이가 0인 경우 temp_n을 기존데이터에서 삭제합니다. 그리고 1.으로 돌아가 반복작업을 시작합니다.

In [3]:
# Train 
temp = pd.DataFrame()
# 초기 시작값(V95가 가장 큰)을 정함
train_trans2 = train_trans
train_trans = train_trans2[(train_trans2['V95'].notnull() ) & (train_trans2['V126'].notnull() ) ].reset_index(drop = True)
#train_trans.reset_index(drop = True, inplace = True)
train_trans = train_trans[['V95', 'V126','TransactionAmt','TransactionID']]
train_trans['client_id'] = 0
temp_n = train_trans[train_trans['V95'] == train_trans['V95'].max()]  
temp_n = temp_n.iloc[[0]] 
temp_n

k = 0
while k < 1000 : # 코드 리팩토링하면 숫자를 더 늘릴 예정
    k = k + 1
    if k % 100 == 0 :
        print(k, "개 행 진행 중")
    # print(temp_n['V95'].values) 
    temp_n_1 = train_trans[train_trans['V95'] == float(temp_n['V95']-1)] # n-1 후보 선택
    
    # temp_n_1 길이가 0인 경우 -> temp_n값 이동하고 끝내기
    if temp_n_1.shape[0] == 0 :
        temp = temp.append(temp_n) # 새 데이터프레임에 추가
        train_trans = train_trans.drop(train_trans.index[temp_n.index]).reset_index(drop=True) # 기존 데이터 프레임에서 삭제
        temp_n = train_trans[train_trans['V95'] == train_trans['V95'].max()] 
        temp_n = temp_n.iloc[[0]]
        train_trans['client_id'] = train_trans['client_id'] + 1 
        #print('Client_id : ' ,float(temp_n['client_id'].values))
        #print("해당하는 값이 없음")
        continue
    
    # tmep_n_1 길이가 1 이상인 경우 매칭되는 값을 탐색 -> 매칭 후 temp 는 temp_n_1로 대체
    for i in range(temp_n_1.shape[0]):
        te = temp_n_1.iloc[[i]]
        if (  round(temp_n['V126'].values[0] - te['V126'].values[0], 1)  )== round(te['TransactionAmt'].values[0], 1) :
            temp = temp.append(temp_n) # 새 데이터프레임에 추가
            train_trans = train_trans.drop(train_trans.index[temp_n.index]).reset_index(drop=True) # 기존 데이터 프레임에서 삭제
            temp_n = te # te로 temp_n을 대체
            #print("=======")
            break
           
        # 모든 값을 탐색했으나 매칭이 되지 않는 경우
        if i == (temp_n_1.shape[0]-1) :
            temp = temp.append(temp_n) # 새 데이터프레임에 추가
            train_trans = train_trans.drop(train_trans.index[temp_n.index]).reset_index(drop=True) # 기존 데이터 프레임에서 삭제
            temp_n = train_trans[train_trans['V95'] == train_trans['V95'].max()] 
            temp_n = temp_n.iloc[[0]]
            train_trans['client_id'] = train_trans['client_id'] + 1
            #print('Client_id : ' ,float(temp_n['client_id'].values))
            #print("매칭 불가")
            break    

100 개 행 진행 중
200 개 행 진행 중
300 개 행 진행 중
400 개 행 진행 중
500 개 행 진행 중
600 개 행 진행 중
700 개 행 진행 중
800 개 행 진행 중
900 개 행 진행 중
1000 개 행 진행 중


In [4]:
print("train client_id 추출값 : ", temp['client_id'].nunique())

train client_id 추출값 :  34


### 결과 확인

In [5]:
ex = pd.merge(train_trans2, temp[['TransactionID','client_id']], on = 'TransactionID', how = 'right')
ex['day'] = round(ex['TransactionDT']/60/60/24,0)
ex['hour'] = round(ex['TransactionDT']/60/60 %24,0)

ex2 = ex.groupby("client_id").agg({'isFraud':['count','mean'], 'TransactionAmt':['nunique'], 'card1':['nunique'],'DeviceInfo':['nunique'],\
                                  'day':['min','max','nunique']}).reset_index()
ex2.columns = ['client_id','transaction_count','isFraud_ratio','TransactionAmt_unique','card1_unique','device_unique','first_day','last_day','unique_day']
ex2 = ex2.sort_values('transaction_count', ascending = False)
ex2.head(10)

,client_id,transaction_count,isFraud_ratio,TransactionAmt_unique,card1_unique,device_unique,first_day,last_day,unique_day
0,0,456,0,6,1,1,172.0,173.0,2
15,15,233,0,4,1,1,166.0,172.0,2
26,26,148,0,4,1,1,165.0,173.0,2
33,33,56,0,3,1,1,172.0,172.0,1
24,24,27,0,2,1,1,165.0,165.0,1
1,1,16,0,2,1,1,173.0,173.0,1
16,16,11,0,2,2,2,165.0,166.0,2
29,29,6,0,2,1,1,166.0,173.0,2
12,12,6,0,1,1,1,173.0,173.0,1
17,17,4,0,2,1,2,166.0,166.0,1


- TranscationAmt, card1, device, day가 다르더라도 묶이는것을 볼 수 있으며, 따라서 잠재적인 client_id라고 추정해볼수있을것 같음

In [ ]:
# Test
temp = pd.DataFrame()
# 초기 시작값(V95가 가장 큰)을 정함
test_trans2 = test_trans
test_trans = test_trans2[(test_trans2['V95'].notnull() ) & (test_trans2['V126'].notnull() ) ].reset_index(drop = True)
#train_trans.reset_index(drop = True, inplace = True)
test_trans = test_trans[['V95', 'V126','TransactionAmt','TransactionID']]
test_trans['client_id'] = 0
temp_n = test_trans[test_trans['V95'] == test_trans['V95'].max()]  
temp_n = temp_n.iloc[[0]] 
temp_n

k = 0
while k < 500 : 
    k = k + 1
    if k % 100 == 0 :
        print(k, "개 행 진행 중")
    # print(temp_n['V95'].values) 
    temp_n_1 = test_trans[test_trans['V95'] == float(temp_n['V95']-1)] # n-1 후보 선택
    
    # temp_n_1 길이가 0인 경우 -> temp_n값 이동하고 끝내기
    if temp_n_1.shape[0] == 0 :
        temp = temp.append(temp_n) # 새 데이터프레임에 추가
        test_trans = test_trans.drop(test_trans.index[temp_n.index]).reset_index(drop=True) # 기존 데이터 프레임에서 삭제
        temp_n = test_trans[test_trans['V95'] == test_trans['V95'].max()] 
        temp_n = temp_n.iloc[[0]]
        test_trans['client_id'] = test_trans['client_id'] + 1 
        #print('Client_id : ' ,float(temp_n['client_id'].values))
        #print("해당하는 값이 없음")
        continue
    
    # tmep_n_1 길이가 1 이상인 경우 매칭되는 값을 탐색 -> 매칭 후 temp 는 temp_n_1로 대체
    for i in range(temp_n_1.shape[0]):
        te = temp_n_1.iloc[[i]]
        if (  round(temp_n['V126'].values[0] - te['V126'].values[0], 1)  )== round(te['TransactionAmt'].values[0], 1) :
            temp = temp.append(temp_n) # 새 데이터프레임에 추가
            test_trans = test_trans.drop(test_trans.index[temp_n.index]).reset_index(drop=True) # 기존 데이터 프레임에서 삭제
            temp_n = te # te로 temp_n을 대체
            #print("=======")
            break
           
        # 모든 값을 탐색했으나 매칭이 되지 않는 경우
        if i == (temp_n_1.shape[0]-1) :
            temp = temp.append(temp_n) # 새 데이터프레임에 추가
            test_trans = test_trans.drop(test_trans.index[temp_n.index]).reset_index(drop=True) # 기존 데이터 프레임에서 삭제
            temp_n = test_trans[test_trans['V95'] == test_trans['V95'].max()] 
            temp_n = temp_n.iloc[[0]]
            test_trans['client_id'] = test_trans['client_id'] + 1
            #print('Client_id : ' ,float(temp_n['client_id'].values))
            #print("매칭 불가")
            break    

한계점

- 코드가 너무더럽다...(아직 파이썬이 익숙하지않은...) 특히 중간의 temp_n_1 길이가 1이상인 코드 파트는 시간이 지날수록 값이 기하급수적으로 늘어나는 경향이있음.. (ex 500이라는 값은 전체 데이터중 5개정도밖에 안되지만 0인 값은 50000개이므로 50000개를 다 탐색해야한다..) 좀더 효율적으로 코드를 짤수있도록 고민하자(현우님이나 순환님께 코드 설명드리고 리팩토링 요청도 드리기)
- 만약 이게 client_id라면 효과가 있어야지..? client id들의 특성을 파악
- 현재 상태에서는 train/test 개별적인 client들만 파악할수있음. train/test를 아우르는 client_id를 파악할 필요